In [ ]:
import pandas as pd
import numpy as np
import re

from google.colab import drive
drive.mount('/content/drive')

import matplotlib.pyplot as plt
import seaborn as sns

Mounted at /content/drive


In [ ]:
import string
import re

In [ ]:
%%capture
!python3 -m spacy download lt_core_news_md

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/data/rek_su_aprasymais.csv')
df.drop(['has_desc'], axis=1, inplace=True)

In [ ]:
df = df.drop_duplicates(subset=['imones_aprasymas'])

In [ ]:
def clean_company_names(x):
  x = x.lower()
  table = str.maketrans("","", string.punctuation)
  x = x.translate(table).lower().split()
  x = ' '.join(x)

  str_to_remove = ['&rdquo', '&bull','&amp']
  for i in str_to_remove:
    x= x.replace(i, '')

  x = re.sub(' +', ' ', x)
  return x

df['imones_pavadinimas'] = df.apply(lambda row: clean_company_names(row['imones_pavadinimas']), axis=1)

In [ ]:
import spacy
import snowballstemmer
nlp = spacy.load("lt_core_news_md")


def basic_clean(x, y):
  # Text to lowercase
  x = x.lower()

  # Remove websites
  pattern = r'(https?://)?(www\.)?[\w\s]+\.\w+'
  x = re.sub(pattern, '', x)

  # Removing particular artifacts
  str_to_remove = ['&rdquo', '&bull','&amp', '[email&#160;protected]', 'kontaktai',
                   'mob.', 'tel.', 'el.', 'paštas', 'telefonu', 'telefonai']
  for r in str_to_remove+y.split(" "):
    x = x.replace(r, '')

  # Remove all digits
  x = re.sub('[0-9]', '', x)

  # Removing all punctuations
  table = str.maketrans("","", string.punctuation + "„“‘–")
  x = x.translate(table)

  # Removing stopwords
  doc = nlp(x)
  x_filtered = [token.text for token in doc if not token.is_stop]
  x_filtered = [word for word in x_filtered if len(word) > 2]
  x = ' '.join(x_filtered)

  # Removing additional empty spaces
  x = re.sub(' +', ' ', x)

  # Stemming
  #stemmer = snowballstemmer.stemmer('lithuanian')
  #x = ' '.join(stemmer.stemWords(x.split()))
  return x.strip()

In [ ]:
df['imones_aprasymas'] = df.apply(lambda row: basic_clean(row['imones_aprasymas'], row['imones_pavadinimas']), axis=1)

In [ ]:
df['desc_len'] = df['imones_aprasymas'].str.count(r'\w+')

In [ ]:
df.sample(5)

,imones_kodas,imones_pavadinimas,imones_aprasymas,desc_len
22349,303080976,jutus uab,autoservisas ratų montavimas ratų balansavimas...,37
7275,190459552,tauragės tarailių progimnazija,mokoma klasės,2
18890,147020459,deimulė uab,korpusiniai minkšti baldai projektavimas gamyb...,13
1135,301645640,waristi uab,įmonė veikianti nidalingua vardu specializuoja...,25
20673,141929550,l rubcovos iį,taka rekama rekamos agentūra įkūrta metais aik...,30


In [ ]:
import seaborn as sns

In [ ]:
df.to_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/parsed_data/semi_clean.csv', index=False)

In [ ]:
df = df[(df['desc_len']>9) & (df['desc_len']<161)]
df.to_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/parsed_data/semi_clean_095.csv', index=False)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/parsed_data/semi_clean.csv')

In [ ]:
df.dropna(axis=0, inplace=True)

In [ ]:
from collections import Counter

In [ ]:
df = df[~df['imones_aprasymas'].str.contains('viliošių')]

In [ ]:
to_remove = ['įmonė', 'yra', 'lietuvoje', 'imonės', 'lietuvos', 'nuolat', 'metais', 'daugiau',
             'tiek', 'esame', 'metų', 'todėl', 'įsikūrusi', 'galime', 'ivairių', 'veiklą', 'www', 'rajone',
             'metu', 'pasiūlyti', 'gali', 'klausimais', 'patirtį', 'siūlome', 'teikia', 'europos', 'visoje',
             'kaune', 'atliekame', 'rasti', 'siekiant', 'vilniuje', 'teikiame', 'dartuotojų',
             'turi', 'tikslas', 'būti', 'bendrovė', 'pasitikėjimą', 'visų', 'šalyse', 'komanda', 'visus',
             'geriausius', 'požiūris', 'leidžia', 'baltijos', 'laikotarpį', 'dirbame', 'užsienio', 'sėkmę',
             'pelnė', 'siekdama', 'taptų', 'sėkmingai', 'buvo', 'vizijos', 'tobulėjo', 'tobulėja', 'taka',
             'įkurta', 'realiais', 'turime', 'atlieka', 'galima', 'garantavo', 'šalių', 'visą', 'siūlo',
             'galite', 'visos', 'visada', 'rasite', 'europoje', 'metus', 'greitai', 'vilniaus', 'siekiame',
             'daug', 'kompanija', 'teikiamos', 'vilnius', 'labai', 'kainą', 'val', 'čia', 'visi', 'vykdo',
             'teikianti', 'iso','atlikti', 'miesto', 'sudaro', 'kartu', 'laiką', 'bendrovės', 'pagrindinės',
             'kauno', 'specializuojasi', 'klaipėdoje', 'užsienyje', 'pagrindinis', 'suteikti', 'užsiima',
             'bus', 'latvijoje', 'įvairaus', 'and', 'šiandien', 'naudojant', 'kuo', 'gerai', 'adresas',
             'asociacijos', 'klaipėdos', 'kompanijos', 'vietą', 'lietuva', 'gauti', 'repusblikos', 'tapti', 'grupė',
             'geriausią', 'bendradarbiaujame', 'visiems', 'atliekami', 'atstovauja', 'atstovaujame', 'komandos', 'mieste', 'bull',
             'darbas', 'vieną', 'estijoje', 'pan', 'visais', 'apima', 'dirbti', 'dirbanti', 'pagaminti', 'viso', 'the',
             'jav', 'group', 'httpwww', 'vizija', 'nėra', 'tapo', 'ilgametė', 'didelį', 'klausimus', 'lietuvą', 'panevėžyje',
             'parą', 'emailprotected', 'klaipėda', 'middot', 'viskas', 'skandinavijos', 'mini', 'buv', 'euro', 'com', 'svarbu',
             'vokietijoje', 'nvs', 'facebook', 'kasdien', 'httpwww', 'pasižymi', 'viską', 'rytų', 'rinkai', 'liebherr', 'eur',
             'google', 'tampa', 'ministerijos', 'nori', 'vsdfv', 'šilainių', 'del', 'ime', 'osstem', 'įmoių', 'ainių','plots',
             'velcro', 'rolls', 'udiniis', 'royce', 'vihrovas', 'uku', 'prizinių', 'atvas', 'etools', 'compac', 'duktų', 'duktus',
             'duktai', 'ėlės', 'self', 'sierra']

In [ ]:
for k,v in counter.most_common():
      if v < 3:
        to_remove.append(k)

In [ ]:
with open("abc", 'w') as f:
    for k,v in  counter.most_common():
        f.write( "{} {}\n".format(k,v) )

In [ ]:
to_remove = []

with open('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/abc(2)', 'r') as f:
  for line in f:
    to_remove.append(line.split()[0])

In [ ]:
def remove_(x):
  x = x.split()
  x = [i for i in x if i not in to_remove]
  x = ' '.join(x)
  x = re.sub(' +', ' ', x)
  return x

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/parsed_data/cleaned_corpus.csv')
df = df.dropna(subset=["imones_aprasymas"])
df = df[~df['imones_aprasymas'].str.contains('viiošiai')]
df = df[~df['imones_aprasymas'].str.contains('viiošių')]
df['imones_aprasymas'] = df.imones_aprasymas.apply(lambda x: remove_(x))

In [ ]:
df['desc_len'] = df['imones_aprasymas'].str.count(r'\w+')

In [ ]:
df = df[~(df.desc_len<2)]

In [ ]:
df.to_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/parsed_data/description_dataset.csv', index=False)

In [ ]:
import pickle

with open('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/parsed_data/additional_stopwords', 'wb') as fp:
    pickle.dump(to_remove, fp)

In [ ]:
df['imones_aprasymas'] = df.imones_aprasymas.apply(lambda x: remove_(x))

In [ ]:
df.to_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/parsed_data/cleaned_corpus.csv', index=False)

In [ ]:
corpus = []
new = df['imones_aprasymas'].str.split().values.tolist()

corpus = [word for i in new for word in i]

counter= Counter(corpus)
counter.most_common(40)